#### Libraries installed

In [1]:
# pip install easygui

### Import libraries

In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.model_selection import train_test_split
import pandas as pd
import zipfile
import easygui 
import sys
import os

### Functions

In [3]:
def get_full_path(download_folder_path , dataset_kaggle_path):
    return fr"{download_folder_path}/{str(dataset_kaggle_path).split('/')[-1]}.zip"

In [4]:
def open_file_dialog(warning = "Please select only CSV file"):
    try:
        return easygui.fileopenbox()
    except:
        return warning

In [5]:
def get_data_frame():
    return pd.read_csv(open_file_dialog())

In [6]:
from easygui import *

def get_text_widget(message , title = "", text = ""):
    return textbox(message, title, text)

In [7]:
from easygui import *

def get_text_choise(choices , text="Please selecte only one item.",title = "Choise box"):
    return choicebox(text, title, choices)

In [8]:
def get_message_box(message , title="Message Box"):
    return msgbox(message, title) 

In [9]:
# scalling
from sklearn.preprocessing import MinMaxScaler
def get_scaled_data(data_frame):
    scaler = MinMaxScaler()
    scal = scaler.fit(data_frame)
    return scal.transform(data_frame)

In [10]:
def get_splitted_data(x , y = None , model_type ="Supervised"):
    if model_type == "Supervised":
#         x = data_frame.drop([data_frame.columns[-1]],axis=1)
#         y = data_frame[data_frame.columns[-1]]

#         x_scale = data_scaling(data_frame.drop([data_frame.columns[-1]],axis=1))

        x_train , x_test , y_train , y_test = train_test_split( x , y , test_size = 0.25 ,  random_state=50,stratify=y)
        return x_train , x_test , y_train , y_test
    elif model_type == "Unsupervised":
        return data_scaling(x)

# Part 1 : Kaggle

### Connectting to Kaggle

In [11]:
api = KaggleApi()
api.authenticate()

### Dataset search

In [12]:
def kagggle_dataset(kaggle_message = "Enter needed dataset name!"):
    # Enter your dataset name --------> GUI
    dataset_needed = get_text_widget(kaggle_message,"Kaggle database","")
    if dataset_needed == ""  or dataset_needed.strip() == "":
        kagggle_dataset(f"There is no empty dataset! \n\nEnter another dataset name!")
        return None
    elif dataset_needed == None:
        return None
    try:    
        datasets_founded_list = api.dataset_list(search = dataset_needed , file_type = "csv")[:]

        all_datasets_founded = ""
        all_datasets_founded_list = list()
        
        for index , dataset_founded in enumerate(datasets_founded_list , 1):
            size = api.datasets_list(search = str(dataset_founded))[0]["totalBytes"]
            all_datasets_founded = f"{str(index).rjust(2,'0')}- {str(datasets_founded_list[index-1]).ljust(70)}  | size -> {(str(size)+' Bytes').rjust(20)} "
            all_datasets_founded_list.append(all_datasets_founded) ##################
        if len(datasets_founded_list) == 0 :
            kagggle_dataset(f"There is no {dataset_needed} \n\nEnter aonther dataset name!")
            return None
        
        index = all_datasets_founded_list.index(get_text_choise(all_datasets_founded_list,title="Kaggle database"))
        
#       code to check if this string is path or not , and handeler if the path was not correct
        selected_path = get_text_widget("Write your download path here!")

        dataset_founded = datasets_founded_list[index]
        print(f'Downloading [{dataset_founded}] dataset')

        dataset_content = api.dataset_list_files(str(dataset_founded)).files

        api.dataset_download_files(str(dataset_founded) , selected_path)
        
        folder = get_text_choise(["YES","NO"] , text = "Do you want to open the directory of the downloaded file?",title = "Q!")
        if folder == "YES":
            os.startfile(selected_path)
        unzip_or_not = get_text_choise(["YES","NO"] , text = "Do you want to unzip the downloaded file?",title = "Q!")
        if unzip_or_not == "YES":
            zip_file_location = get_full_path(selected_path , dataset_founded)
            if unzip_file(selected_path , zip_file_location , dataset_founded):
                get_message_box("Done unzipping successfully")
        
    except:
        print("ERROR404!")

### Extracting downloaded file

In [13]:
def unzip_file(selected_path , zip_file_location , dataset_founded):
    os.chdir(selected_path)
    with zipfile.ZipFile(zip_file_location,'r') as file:
        file.extractall(str(dataset_founded).split("/")[-1])
        return True
    return False

# Part 2 : Machine Learning

### 1- Supervised machine learning models

#### Logistic regression

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

def logistic_regression(x_train_sc , x_test_sc , y_train_sc , y_test_sc):
    model2 = LogisticRegression()
    model2.fit(x_train_sc, y_train_sc)

    train_pred = model2.predict(x_train_sc)
    test_pred = model2.predict(x_test_sc)

    train_score = metrics.accuracy_score( train_pred , y_train_sc )
    test_score = metrics.accuracy_score( test_pred , y_test_sc )

    logistic_regression_score = f"{'Logistic regression score'.center(25)} | {'Train score'.center(15)} -> {round(train_score * 100,3)}% {'Test score'.center(15)} -> {round(test_score * 100,3)}%"
#     scores["Logistic_regression"] = Logistic_regression

    return logistic_regression_score

#### Linear regression

In [15]:
from sklearn.linear_model import LinearRegression

def linear_regression(x_train_sc , x_test_sc , y_train_sc , y_test_sc):
    model = LinearRegression()
    model.fit(x_train_sc, y_train_sc)
    
    predict = model.predict(x_test_sc)
    
    train_score = model.score(x_train_sc , y_train_sc) 
    test_score = model.score(x_test_sc , y_test_sc) 
    logistic_regression_score = f"{'Logistic regression score'.center(25)} | {'Train score'.center(15)} -> {round(train_score * 100,3)}% {'Test score'.center(15)} -> {round(test_score * 100,3)}%"
    return logistic_regression_score

#### KNN

In [16]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics

def k_neighbor(x_train_sc , x_test_sc , y_train_sc , y_test_sc):
    model3 = KNeighborsClassifier()
    model3.fit(x_train_sc, y_train_sc)

    tr_pred = model3.predict(x_train_sc)
    te_pred = model3.predict(x_test_sc)

    train_score = metrics.accuracy_score(tr_pred,y_train_sc)
    test_score = metrics.accuracy_score(te_pred,y_test_sc)

    knn_score = f"{'KNeighbors score'.center(25)} | {'Train score'.center(15)} -> {round(train_score * 100,3)}% {'Test score'.center(15)} -> {round(test_score * 100,3)}%"
#     scores["KNN_score"] = KNN_score
    
    return knn_score

#### Decisiond tree

In [17]:
from sklearn.tree import DecisionTreeClassifier

def decisiond_tree(x_train_sc , x_test_sc , y_train_sc , y_test_sc):
    
    model4 = DecisionTreeClassifier( max_depth = 3)
    model4.fit(x_train,y_train)

    train_score = model4.score( x_train , y_train )
    test_score = model4.score( x_test , y_test )

    decision_tree_score = f"{'Decision tree score'.center(25)} | {'Train score'.center(15)} -> {round(train_score * 100,3)}% {'Test score'.center(15)} -> {round(test_score * 100,3)}%"
    # scores["decision_tree_score"] = decision_tree_score
    
    return decision_tree_score

#### Neural network

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

def neural_network(x_train,x_test, y_train, y_test):
    model5 = MLPClassifier(activation='relu',
                                       solver='adam',
                                       learning_rate='constant',
                                       early_stopping=False,
                                       alpha=0.001, hidden_layer_sizes=(100, 3), random_state = 33)
    model5.fit(x_train, y_train)

    train_score = model5.score(x_train, y_train)
    test_score = model5.score(x_test, y_test)

    neural_network_score = f"{'Neural network score'.center(25)} | {'Train score'.center(15)} -> {round(train_score * 100,3)}% {'Test score'.center(15)} -> {round(test_score * 100,3)}%"
    # scores["neural_network_score"] = neural_network_score
    return neural_network_score

#### SVM

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

def svm(x_train_sc , x_test_sc , y_train_sc , y_test_sc):
    
    param_grid = {'C': [0.1, 1, 10, 100, 1000],
                  'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                  'kernel': ['rbf'] }

    grid = GridSearchCV(SVC(), param_grid)
    grid.fit(x_train_sc, y_train_sc)

    ###################################################

    grid_predictions = grid.predict(x_test_sc)
    metrics.accuracy_score(grid_predictions,y_test_sc)

    ##################################################

    model6 = SVC()
    model6.fit(x_train_sc, y_train_sc)

    train_predictions = model6.predict(x_train_sc)
    test_predictions = model6.predict(x_test_sc)

    model6.get_params()

    train_score = metrics.accuracy_score(train_predictions,y_train_sc) 
    test_score = metrics.accuracy_score(test_predictions,y_test_sc)

    svm_score = f"{'SVM score'.center(25)} | {'Train score'.center(15)} -> {round(train_score * 100,3)}% {'Test score'.center(15)} -> {round(test_score * 100,3)}%"
    return svm_score

#### Random forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

def random_forest(x_train_sc , x_test_sc , y_train_sc , y_test_sc):
    model7 = RandomForestClassifier(n_estimators = 20 , random_state = 30)
    model7.fit(x_train_sc,y_train_sc)

    prediction_test = model7.predict(x_test_sc)
    test_score = metrics.accuracy_score(y_test_sc, prediction_test)

    random_forest = f"{'Random forest'.center(25)} | {'Train score'.center(15)} -> ------% {'Test score'.center(15)} -> {round(test_score * 100,3)}%"
#     scores["random_forest"] = random_forest

#     feature_list = list(x.columns)
#     feature_imp = pd.Series(model7.feature_importances_,index=feature_list).sort_values(ascending=False)
    return random_forest

#### Naive baise

In [21]:
from sklearn.naive_bayes import GaussianNB

def naive_baise(x_train,x_test, y_train, y_test):
    model8 = GaussianNB()
    model8.fit(x_train,y_train)

    train_score = model8.score(x_train , y_train)
    test_score = model8.score(x_test , y_test)

    naive_baise_score = f"{'Naive baise'.center(25)} | {'Train score'.center(15)} -> {round(train_score * 100,3)}% {'Test score'.center(15)} -> {round(test_score * 100,3)}%"
    return naive_baise_score

### Unsupervised machine learning models

#### Clustring

In [22]:
# Code


#### Hierarchical clustering

In [23]:
# code

### 

#### Create your data frame

In [24]:
# Load your csv file to create the dataframe directly , If you have not any data frame ready to be used this 
counter = 1
data_frame = None
def load_data_frame():
    try:
        global data_frame
        data_frame = get_data_frame()
        return True
    except:
        global counter
        if counter <= 3:
            counter += 1
            get_message_box("Please select olny preproccessed csv file to avoid errors!")
            load_data_frame()
        else:
            get_message_box("You were warned to select only csv files :) ")
            return False
# We calculate how many you will choose the wrong file, 
# so that I can close the program so that it does not fall into infinity loop

In [25]:
def run():    
    items =[
    "Download kaggle dataset",
    "Use machine learning models"
    ]

    item = get_text_choise(choices = items)
    
    if item == "Download kaggle dataset":
        kagggle_dataset()
    elif item == "Use machine learning models":
        ml_type = get_text_choise(choices=["Supervised","Unsupervised","Back"])
        if ml_type == "Supervised":
#           Check if the user will predict a continous value or descrete to decide what algorithms will be used!
            supervised_models_list = [
                "Logistic regression",
                "Linear regression",
                "KNN",
                "Decisiond tree",
                "Neural network",
                "SVM",
                "Random forest",
                "Naive baise",
                "All models",
                "Back"
            ]
            supervised_model = get_text_choise(supervised_models_list)
            
            y = data_frame[data_frame.columns[-1]]
            x = data_frame.drop(data_frame.columns[-1],axis=1)
            
            x_train , x_test , y_train , y_test = get_splitted_data(x,y)
            
            x = get_scaled_data(x)
            
            x_trainsc , x_testsc , y_trainsc , y_testsc = get_splitted_data(x,y)
            
            try:
                index_of_model = supervised_models_list.index(supervised_model)
                print("index_of_model",index_of_model)
                if index_of_model == 0:
                    score = logistic_regression(x_trainsc , x_testsc , y_trainsc , y_testsc)
                    get_message_box(score)
                    run()
                    return None
                elif index_of_model == 1:
                    score = linear_regression(x_trainsc , x_testsc , y_trainsc , y_testsc)
                    get_message_box(score)
                    run()
                    return None
                elif index_of_model == 2:
                    score = k_neighbor(x_trainsc , x_testsc , y_trainsc , y_testsc)
                    get_message_box(score)
                    run()
                    return None
                elif index_of_model == 3:
                    score = decisiond_tree(x_trainsc , x_testsc , y_trainsc , y_testsc)
                    get_message_box(score)
                    run()
                    return None
                elif index_of_model == 4:
                    score = neural_network(x_train , x_test , y_train , y_test)
                    get_message_box(score)
                    run()
                    return None
                elif index_of_model == 5:
                    score = svm(x_trainsc , x_testsc , y_trainsc , y_testsc)
                    get_message_box(score)
                    run()
                    return None
                elif index_of_model == 6:
                    score = random_forest(x_trainsc , x_testsc , y_trainsc , y_testsc)
                    get_message_box(score)
                    run()
                    return None
                elif index_of_model == 7:
                    score = naive_baise(x_train , x_test , y_train , y_test)
                    get_message_box(score)
                    run()
                    return None
                elif index_of_model == 8:
#                     All accuracy

                    score = logistic_regression(x_trainsc , x_testsc , y_trainsc , y_testsc) + "\n"
                    score += linear_regression(x_trainsc , x_testsc , y_trainsc , y_testsc) + "\n"
                    score += k_neighbor(x_trainsc , x_testsc , y_trainsc , y_testsc) + "\n"
                    score += decisiond_tree(x_trainsc , x_testsc , y_trainsc , y_testsc) + "\n"
                    score += neural_network(x_train , x_test , y_train , y_test) + "\n"
                    score += svm(x_trainsc , x_testsc , y_trainsc , y_testsc) + "\n"
                    score += random_forest(x_trainsc , x_testsc , y_trainsc , y_testsc) + "\n"
                    score += naive_baise(x_train , x_test , y_train , y_test)
                    print(score)
                    
#                     Concatination in jupyter does not work!!!!!!
                    
                    get_message_box(score)
                    run()
                    return None
                elif index_of_model ==10:
                    run()
                    return None
                else:
                    get_message_box("The program will be closed :) ")
                    sys.exit()
                
            except:
#                 If any errors happened then the user closed the window , so we're gonna close the programe
                exit()                
        elif ml_type == "Unsupervised":
            unsupervised_models_list = [
                "No models avilable now",
                "Back"
            ]
            unsupervised_model = get_text_choise(unsupervised_models_list)
        elif ml_type == "Back":
            run()
            return None
        else:
            return None
    else:
        return None

#### RUN

In [33]:
if (load_data_frame()):
    run()
else:
    get_message_box("The program will be closed :) ")

ERROR404!


In [27]:
# Datasets for testing

In [28]:
# # Enginner Asmaa csv file

# data_frame = pd.read_csv("diabetes.csv")
# y = data_frame["Outcome"]
# x = data_frame.drop("Outcome",axis=1)

# # Before scaling

# x_train , x_test , y_train , y_test = get_splitted_data(x,y)

# print(logistic_regression(x_train , x_test , y_train , y_test))
# print(linear_regression(x_train , x_test , y_train , y_test))
# print(k_neighbor(x_train , x_test , y_train , y_test))
# print(decisiond_tree(x_train , x_test , y_train , y_test))
# print(neural_network(x_train , x_test , y_train , y_test))
# print(svm(x_train , x_test , y_train , y_test))
# print(random_forest(x_train , x_test , y_train , y_test))
# print(naive_baise(x_train , x_test , y_train , y_test))


# print(120*"-")

# After scaling

# x = get_scaled_data(x)

# x_trainsc , x_testsc , y_trainsc , y_testsc = get_splitted_data(x,y)

# print(logistic_regression(x_train , x_test , y_train , y_test))
# print(linear_regression(x_train , x_test , y_train , y_test))
# print(k_neighbor(x_train , x_test , y_train , y_test))
# print(decisiond_tree(x_train , x_test , y_train , y_test))
# print(neural_network(x_train , x_test , y_train , y_test))
# print(svm(x_train , x_test , y_train , y_test))
# print(random_forest(x_train , x_test , y_train , y_test))
# print(naive_baise(x_train , x_test , y_train , y_test))

In [29]:
# Another test

In [30]:
# # Engineer Rawan csv file 

# data_frame = pd.read_csv("titanic400.csv")
# data_frame.drop(columns=["Name","PassengerId","Cabin","Ticket","Parch","Embarked","SibSp"],inplace=True)
# data_frame["Age"].fillna(data_frame["Age"].mean(),inplace=True)
# data_frame["Sex"].replace({ "female":0 , "male":1 } , inplace=True)
# data_frame.dropna(axis="index",how="any",inplace=True)

# x = data_frame.drop(["Survived"] , axis = 1)
# y = data_frame['Survived']


# x_train , x_test , y_train , y_test = get_splitted_data(x,y)

# print(logistic_regression(x_train , x_test , y_train , y_test))
# print(linear_regression(x_train , x_test , y_train , y_test))
# print(k_neighbor(x_train , x_test , y_train , y_test))
# print(decisiond_tree(x_train , x_test , y_train , y_test))
# print(neural_network(x_train , x_test , y_train , y_test))
# print(svm(x_train , x_test , y_train , y_test))
# print(random_forest(x_train , x_test , y_train , y_test))
# print(naive_baise(x_train , x_test , y_train , y_test))



# x = get_scaled_data(x)


# # After scaling

# x_trainsc , x_testsc , y_trainsc , y_testsc = get_splitted_data(x,y)

# print(logistic_regression(x_train , x_test , y_train , y_test))
# print(linear_regression(x_train , x_test , y_train , y_test))
# print(k_neighbor(x_train , x_test , y_train , y_test))
# print(decisiond_tree(x_train , x_test , y_train , y_test))
# print(neural_network(x_train , x_test , y_train , y_test))
# print(svm(x_train , x_test , y_train , y_test))
# print(random_forest(x_train , x_test , y_train , y_test))
# print(naive_baise(x_train , x_test , y_train , y_test))


In [31]:
# # I saved this file after preproccessing to test the code 

# # This is Engineer Rawan file

# data_frame = pd.read_csv("titanic400.csv")
# data_frame.drop(columns=["Name","PassengerId","Cabin","Ticket","Parch","Embarked","SibSp"],inplace=True)
# data_frame["Age"].fillna(data_frame["Age"].mean(),inplace=True)
# data_frame["Sex"].replace({ "female":0 , "male":1 } , inplace=True)
# data_frame.dropna(axis="index",how="any",inplace=True)
# kk = data_frame["Survived"]
# data_frame.drop("Survived",axis=1,inplace=True)
# data_frame["Survived"] = kk
# data_frame

In [32]:
##  save csv file
# data_frame.to_csv("E:\Courses\IEEE\preproccessed_titanic.csv")